**Run the codes in this sequence: A, B, C, D, E, F, G, H, I, J, K.**

# (A) Import libraries

In [ ]:
# os.chdir(r'C:/Users/UWAdmin/Desktop/TEST2/line codes')

from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01


In [ ]:
import time 
from datetime import timedelta

import rpy2.robjects as robjects

from newportxps import NewportXPS #motion controller
from thread import MotionThread, MirrorThread
import os

import numpy as np
import pandas as pd
import csv
from write_utils import write_data_files, write_more, repeats, take_mean

from flipper import mirror #import flipper mirror codes
from spectra import capture_photo #import spectrometer codes

# (B) Activate motion controller

In [ ]:
xpsd_remoteip = '192.168.254.254'

xps = NewportXPS(xpsd_remoteip)
print(xps.status_report())

#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        

xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

## (C) Switch on the spectrometer

In [ ]:
#Before running this code check that AddInProcess.exe is running (closed if open)
#Close anything related to Light Field

capture_photo("start",2,1,0) # the values 2,1,0 don't matter, they are later used for indexing tasks

## (D) Import pressure controls
#Used to input new Sample
#Move this before auto focusing

In [ ]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum, quick_fill
current_pressure()

In [ ]:
t = 0.1 # in s
to_ambient()
time.sleep(1)
to_ambient()
to_vacuum(t)
quick_fill()
time.sleep(1)
to_vacuum(t)


# (E) To make sure that the material gets laser beam of max. intensity , focusing of z axis needs to be properly adjusted. this is done by adjust()

In [ ]:
mirror('on')

#Iterate for input to find z value
print("Enter z value (enter 0 when done):")
z=''
while z != 0:
    if z == '':
        z = float(input(':'))
    print(z)
    xps.move_stage('XYZ.Z', z)
    xps.move_stage('XYZ.X', 5)
    xps.move_stage('XYZ.Y', 7)
    z = float(input(':'))
    

# mirror('on')
pos_all()
while input('Is LightField Scan Stopped?:(yes/no)')!='yes':
    continue

# (F) Prepare initial dataset

In [ ]:
from pathlib import Path
path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')
write_data_files(path, series=2)

# (G) Defining threads

-Thread for Mirror 

-Thread for switching the mirror on and off

In [ ]:
# #Change this to motion_thread
# class Aa(Thread):
#     """MOTOR THREAD"""
#     def run(self):
#         import pandas as pd
#         d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
#         d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
#         d.columns = ['ramptime','rampdist','rampvel']
#         total_time=np.sum(d['ramptime'])
        
#         if d['ramptime'][0]<0.5:
#             time.sleep(.5-d['ramptime'][0])
#         print("MOTION THREAD\n")
#         print("XPS run trajectory")
#         xps.run_trajectory('foreward',)
#         print('Total time {} in class'.format(total_time))
#         time.sleep(total_time)
#         print("TOTAL_TIME DONE, LASER 0.05W")
#         laser_power.power=0.01
#         print("finished and current position is:\n")
#         pos_all()

# #Change this to mirror_thread
# class Bb(Thread):
#     """MOTOR THREAD"""
#     def run(self):
#         import pandas as pd
#         d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
#         d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
#         d.columns = ['ramptime','rampdist','rampvel']
#         mirror_sleep=d['ramptime'][0]-.5
#         print("MIRROR THREAD")
#         for i in range(2):
#             if i==0:
#                 print(d['ramptime'][0])
#                 if d['ramptime'][0]<0.5:
#                     print("mirror on")
       
#                     start_time = time.monotonic()
#                     mirror('on')
       
#                     end_time = time.monotonic()
#                     print(timedelta(seconds=end_time - start_time))
#                     time.sleep(d['ramptime'][1]) #time for linear line    

#                 else:
#                     print('{} > 0.5'.format(d['ramptime'][0]))
#                     print('Mirror sleep {} in class'.format(mirror_sleep))
#                     time.sleep(mirror_sleep)
                
#                     print("mirror on")
#                     start_time = time.monotonic()
#                     mirror('on')
#                     end_time = time.monotonic()
#                     print(timedelta(seconds=end_time - start_time))
#                     time.sleep(d['ramptime'][1]) #time for linear line
#             else:
#                 mirror('off')
#                 print("mirror off")

# (H) Define coordinate details

**Takes input for**

-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

In [ ]:
lines ,startx, starty, step_y = 7, 2.5, 6.5, 1

print(lines,startx,starty,step_y)
move_y = [0 for i in range(int(lines))]


for i in range(int(lines)):
#     print(i)
    if i == 0: move_y[i] = float(starty)
    if i > 0 :
        move_y[i] = float(move_y[i-1]) + float(step_y)
#         print(move_y[i-1]+float(step_y))


print("Lines will be made at following y axis:",move_y,"\n")


# (K) Begin the experiment

In [ ]:
df2=pd.read_csv('data.csv')
print("Pateerning is to be started\n")
import time
time.sleep(5)
cc=0
steps=0
for i in range(8):
    print("Line number: ",i)
    line=i
    
    #move the axes to their start position
    
    xps.move_stage('XYZ.Pos2',move_y[i])
    pos_all()
    print("\n\n")
    df2=pd.read_csv('data.csv')
    time_of_file=df2['time'][i]
    #scantime=time_of_file/1000
    #define the trajectory
    
    xps.define_line_trajectories(start=float(startx),
                             stop=float(startx)+2,
                             step=0.01,
                             scantime=(time_of_file/1000),
                             axis='Pos1')
    
    print("\nCurrent time is:",time_of_file)
        
    pressure_of_file=df2['pressure'][i]
    
    
    for j in range(3):
        gopr(pressure_of_file)
        close_all()
        time.sleep(5)
    print("\n Pressure is now : ",pressure_of_file,"\n")
    
#     k+=3
    xps.download_trajectory('foreward.trj')
    import pandas as pd
    d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
    d = d.loc[:, (d != 0).any(axis=0)]
    d.columns = ['ramptime','rampdist','rampvel']

   
    
    
    total_time=np.sum(d['ramptime'])
    mirror_sleep=d['ramptime'][0]-.5
    
    
    #power will be set to assigned value and make sure mirror is off and power is given enough time to reach it's value
   
    power_of_file=df2["power"][i]
    ij=3
  

##################

    print("pre-pattern checking\n")
    check=[(float(startx)+.4),float(startx)+.8,(float(startx)+1.2)]
    mm=0
    for kk in range(3):
        xps.move_stage('XYZ.Pos1',check[kk])
        print(pos_all())
        
        for iii in range(3):
            mirror("on")
            #writing G/D of 3 spots in 3 lines inside dataset.csv
    #         capture_photo("on",kk,line)
            print("hey")
            GD=capture_photo("first",kk,line,iii)

            mirror("off")
            time.sleep(5)
    
 
#################### 

#experiment starts after pre-checking

    print("power is now:",power_of_file)
    xps.move_stage('XYZ.Pos1',(float(startx)-d['rampdist'][0]))
    pos_all()
    time.sleep(15)
    print("\n\n")
    mirror("off")
    a=power_of_file
    laser_power.power=((a-2.7979)/0.4639)/1000
    time.sleep(15)
    
    ##Draw Lines
    time.sleep(5)
    
    
    
    
    import sys
    a=Aa()
    b=Bb()
    # stop_threads = False
    a.start()
    b.start()
    
    a.join()
    b.join()
    time.sleep(10)
    print("\n Job done\n")
    
    
    
    ##raman spectra analysis
    laser_power.power=0.01
    time.sleep(15)
    xps.move_stage('XYZ.Pos1',6.5)
    xps.move_stage('XYZ.Pos2',9.5)
    xps.move_stage('XYZ.Pos3',9.3)
          
########################################

#post patterning raman analysis

    check=[(float(startx)+.4),float(startx)+.8,(float(startx)+1.2)]
    mm=0
    for kk in range(3):
        xps.move_stage('XYZ.Pos1',check[kk])
        print(pos_all())
        
        for iii in range(3):
            mirror("on")
            #writing G/D of 3 spots in 3 lines inside dataset.csv
    #         capture_photo("on",kk,line)
            print("hey")
            GD=capture_photo("on",kk,line,iii)

            mirror("off")
            time.sleep(5)      
            
    if i<=3:
        take_median(steps,line)
        steps+=9
        
    
    if i>3:
        mv=take_median(steps,line)
        steps+=9
        if mv is None:
            rsum((0))

        else:
            mv=float(mv)
            rsum((mv))
        write_more()
        repeats()
    

    if i>=3:
        print("AI started thinking")
        robjects.r('''
                sourin<-function(s){

                if(s==1){
                require(XML)

                  suppressWarnings({suppressMessages({
                    library(mlrMBO)
                    library(ggplot2)
                    library(doMC)
                  })})

                  ps = makeParamSet(
                    makeIntegerParam("power", lower = 151, upper =1500),
                    makeIntegerParam("time", lower = 2000, upper = 3000),
                    makeIntegerParam("pressure", lower = 100, upper = 130)

                  )

                  ctrl = makeMBOControl(y.name = "ratio")
                  ctrl = setMBOControlInfill(ctrl, opt = "focussearch",opt.focussearch.maxit = 10, opt.focussearch.points = 10000, crit =makeMBOInfillCritCB(cb.lambda=1))

                #dataset-2 should be replaced and later this line as well
                  data=read.csv("data.csv")

                data<-na.omit(data)
                  suppressMessages({opt.state = initSMBO(par.set = ps,design = data, control = ctrl, minimize = FALSE, noisy = TRUE)})

                  print("Proposed parameters:")
                  prop <- suppressWarnings({proposePoints(opt.state)})
                  print(prop$prop.points) 
                  print(prop$crit.components)
                  saveRDS(prop,'./prop.rds')
                  saveRDS(opt.state,'./opt.state.rds')
                  dummy_ratio=0
                  cat("Expected value (upper bound):\n")
                  cat(paste(prop$crit.components$mean, " (", \
                  prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

                  x<-c(prop$prop.points,dummy_ratio,prop$crit.components$mean,\
                  prop$crit.components$mean + prop$crit.components$se)

                  x2<-c(prop$prop.points)
                  #data_plot=read.csv("plot_data.csv")


                  #dataset-2 should be replaced and later this line as well
                  write.table(x2, file = "data.csv", sep = ",",\
                  append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                  #write.table(x, file = "plot_data.csv", sep = ",",\
                  #append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                }
                else{
                  prop=readRDS('./prop.rds')
                  opt.state=readRDS('./opt.state.rds')
                  library(mlrMBO)
                  print(prop$crit.components)
                  print(prop$prop.points)
                  print("Proposed parameters:")
                  updateSMBO(opt.state, x = prop$prop.points, y = s)
                  prop <- suppressWarnings({proposePoints(opt.state)})
                  print(prop$prop.points) 
                  print(prop$crit.components)
                  saveRDS(prop,'./prop.rds')
                  saveRDS(opt.state,'./opt.state.rds')
                  dummy_ratio=0
                  cat("Expected value (upper bound):\n")
                  cat(paste(prop$crit.components$mean, " (", prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
                  x<-c(prop$prop.points,dummy_ratio,prop$crit.components$mean,prop$crit.components$mean + prop$crit.components$se)
                  x2<-c(prop$prop.points)
                  #data_plot=read.csv("plot_data.csv")

                  #dataset-2 should be replaced and later this line as well
                  write.table(x2, file = "data.csv", sep = ",", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
                  #write.table(x, file = "plot_data.csv", sep = ",", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                }}

                ''')



    if i==3: ##AI wil start working when initial design has 4 data (0,1,2,3= 4 data)
        rsum=robjects.r['sourin']
        rsum((1))
        write_more()
        repeats()



# (I) Parameters predicted by BO is written on data.csv. This is further written on dataset.csv and dataset-pre.csv using write_more() function

In [ ]:
def write_more() -> None:
    """
    Used after doing the AI stuff
    """
    d = pd.read_csv('data.csv')
    ln = d.shape[0]

    vpower = d['power'][ln-1]
    vtime = d['time'][ln-1]
    vpressure = d['pressure'][ln-1]

    d1 = pd.read_csv('dataset.csv')
    ln = d1.shape[0]
    d1.loc[ln,"power"] = vpower
    d1.loc[ln,"time"] = vtime
    d1.loc[ln,"pressure"] = vpressure
    d1.to_csv('dataset.csv',index = False)
    d1.to_csv('dataset-pre.csv',index = False)

# (J) Each line has 3 spots to check and each spot 3 times. so in the csv 9 rows of data for each line

**This is done using repeats() function**

In [ ]:
def repeats() -> None:
    # d1 = pd.read_csv('data.csv')
    df2 = pd.read_csv('dataset.csv')
    ln = len(df2['power'])
    m = ln
    print(f"DATASET LENGTH: {ln}")
    counter  = m
    for i in range(8):
        toAdd = [df2['power'][m-1],df2['time'][m-1],df2['pressure'][m-1]]
        filename = "dataset.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter+1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)
                
    for i in range(8):
        toAdd = [df2['power'][m - 1],df2['time'][m - 1],df2['pressure'][m - 1]]
        filename = "dataset-pre.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter + 1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)

# (J) For the 9 readings of a single line, mean is taken. it's done by take_mean() function

In [ ]:
def take_mean(steps: float, save_line: int) -> float:
    
    print(os.getcwd())
    df = pd.read_csv('dataset.csv')

    valss = np.sort([df['ratio'][steps+8], df['ratio'][steps+7], df['ratio'][steps+6],
                    df['ratio'][steps+5], df['ratio'][steps+4], df['ratio'][steps+3],
                   df['ratio'][steps+2], df['ratio'][steps+1], df['ratio'][steps]])

    lst = [s for s in valss if str(s) != 'nan']
    result = np.mean(lst)
    df2 = pd.read_csv('data.csv')
    df2.loc[save_line,"ratio"] = result
    df2.to_csv('data.csv',index = False)
    return result